## 4. <a id='4_cell'></a>Create Group using Excel, Pandas and the PyFHIR models....

This uses Pandas to easily upload patient data from an Excel (or CSV) file based
on some tricks I learned to create instancea fo resources a lot more complex than patient such as conformance
resources like CapabilityStatement and StructureDefinitition.

- create group resources for combinations of:

1. all patients
1. all patients + timeframe ( today, tomorrow )
1. all patients + status
1. practitioner
1. practitioner + timeframe ( today, tomorrow )
1. practitioner + status
1. location
1. location + timeframe ( today, tomorrow )
1. location + status
1. service
1. service + timeframe ( today, tomorrow )
1. service + status

In [156]:
l = 'http://loinc.org'
rt = 'http://hl7.org/fhir/resource-types'
parameters_map = {
 'all':('LA18187-7', l,'All'),
 'practitioner': ('Practitioner', rt,'Practitioner',"Doctor "),
 'active':('LA16666-2', l,'Active'),
 'discharged':('LA14102-0', l,'Discharged'),
 'location':("Location", rt, 'Location', 'Location: '),
  'service':('HealthcareService', rt, 'HealthcareService', 'Service: ')
    }

parameters = ['active']    #['all','practitioner','active','discharged','location','service']
timeframe = 'today' #[None,'today','tomorrow']
pl_use_case = '-'.join(parameters)
if timeframe:
    pl_use_case = f'{pl_use_case}-{timeframe}'
print(pl_use_case)

active-today


- Then validate and load to ref server
- Then perform searches and print out results - ( print out transactions as markdown )

- Todo using Synthea generated patients and providers


### Load and excel file using Pandas

I've made a sample excel file `my_patients.xlsx` containing data for several patients.
Note that you will need to install the *pandas* module.

- See this set of tutorials to learn more about Pandas:

   [![pandas_tut](pandas.png)](https://www.youtube.com/watch?v=ZyhVh-qRZPA&list=PL-osiE80TeTsWmV9i9c58mdDCSskIFdDS "Pandas Tutorial")

In [157]:
cd ~/pyFHIR_models # This is to make sure in right directory for Binder

[Errno 2] No such file or directory: '/Users/ehaas/pyFHIR_models # This is to make sure in right directory for Binder'
/Users/ehaas/Documents/Python/MyBinder/pyFHIR_models


In [158]:
from pandas import *
df = read_excel('my_patients.xlsx','Sheet1',na_filter = False, dtype=str)
# treat all empty cells as '' and convert all data including booleans to string  

df

,row,my_id,f_name,l_name,gender,b_date,is_deceased,is_active,phone,address1,city,state,zip,Practitioner,Location,HealthCareService,admit_date,discharge_date
0,1,123,Amy,Brown,F,1964-06-19 00:00:00,False,True,5555555555,100 Main St,Napa,CA,94559,smith,Queen,proctology,2020-05-03 00:00:00,2020-05-04 00:00:00
1,2,234,Bert,Black,M,1969-08-09 00:00:00,False,True,5555555555,100 Oak St,Napa,CA,94558,jones,St Helena,urology,2020-05-03 00:00:00,2020-05-04 00:00:00
2,3,345,Colleen,Blue,F,2000-09-21 00:00:00,False,False,5555555555,100 Spruce St,Napa,CA,94559,smith,Queen,dermatology,2020-05-03 00:00:00,2020-05-05 00:00:00
3,4,456,Daniel,Crimson,M,1996-05-23 00:00:00,False,True,5555555555,100 Yahome St,Napa,CA,94558,jones,St Helena,proctology,2020-05-04 00:00:00,2020-05-05 00:00:00
4,5,567,Elizabeth,Green,F,1941-10-21 00:00:00,False,True,5555555555,100 Trancas Blvd,Napa,CA,94559,smith,Queen,urology,2020-05-04 00:00:00,2020-05-05 00:00:00
5,6,689,Frederick,Gray,M,1964-06-26 00:00:00,False,True,5555555555,100 A St,Napa,CA,94558,jones,St Helena,dermatology,2020-05-04 00:00:00,2020-05-05 00:00:00
6,7,890,Greg,Orange,M,1982-05-01 00:00:00,False,True,5555555555,100 Elm St,Napa,CA,94558,smith,Queen,proctology,2020-05-04 00:00:00,2020-05-06 00:00:00
7,8,901,Harriet,Silver,F,2002-02-02 00:00:00,False,True,5555555555,100 E St,Napa,CA,94558,jones,St Helena,urology,2020-05-05 00:00:00,2020-05-07 00:00:00
8,9,902,Imelda,Gold,F,1959-03-19 00:00:00,False,True,5555555555,100 Imola St,Napa,CA,94558,smith,Queen,dermatology,2020-05-05 00:00:00,2020-05-06 00:00:00
9,10,903,Jack,White,M,1988-08-08 00:00:00,False,True,5555555555,200 Trancas Blvd,Napa,CA,94558,jones,St Helena,proctology,2020-05-05 00:00:00,2020-05-06 00:00:00


In [159]:
df.dtypes

row                  object
my_id                object
f_name               object
l_name               object
gender               object
b_date               object
is_deceased          object
is_active            object
phone                object
address1             object
city                 object
state                object
zip                  object
Practitioner         object
Location             object
HealthCareService    object
admit_date           object
discharge_date       object
dtype: object

#### convert admit-date to datetime object

In [160]:
#df['admit_date'] = to_datetime(df['admit_date'])
#df['discharge_date'] = to_datetime(df['discharge_date'])

#### getting min mas and unique values for row

In [161]:
df.admit_date.max(), df.admit_date.min(), getattr(df, "Location").unique(), df.Practitioner.unique(), df.HealthCareService.unique()

('2020-05-05 00:00:00',
 '2020-05-03 00:00:00',
 array(['Queen', 'St Helena'], dtype=object),
 array(['smith', 'jones'], dtype=object),
 array(['proctology', 'urology', 'dermatology'], dtype=object))

### How to Access Dataframe Data Using dot.notation

To do this use the [itertuples method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.itertuples.html).

In [162]:
for i in df.itertuples(index=True):     
    print(f'i.row = {i.row}: i.f_name = {i.f_name}, i.l_name = {i.l_name}, i.gender = {i.gender} , etc...')

i.row = 1: i.f_name = Amy, i.l_name = Brown, i.gender = F , etc...
i.row = 2: i.f_name = Bert, i.l_name = Black, i.gender = M , etc...
i.row = 3: i.f_name = Colleen, i.l_name = Blue, i.gender = F , etc...
i.row = 4: i.f_name = Daniel, i.l_name = Crimson, i.gender = M , etc...
i.row = 5: i.f_name = Elizabeth, i.l_name = Green, i.gender = F , etc...
i.row = 6: i.f_name = Frederick, i.l_name = Gray, i.gender = M , etc...
i.row = 7: i.f_name = Greg, i.l_name = Orange, i.gender = M , etc...
i.row = 8: i.f_name = Harriet, i.l_name = Silver, i.gender = F , etc...
i.row = 9: i.f_name = Imelda, i.l_name = Gold, i.gender = F , etc...
i.row = 10: i.f_name = Jack, i.l_name = White, i.gender = M , etc...


### Create Group Instance for All Patients, One member for Each Row...

1. initialize group resource and datatype instances
1. iterate over dataframe rows
1. assign dataframe data to member elements (may need to massage the strings to fit the data-types - see `active` for example)
1. validate and save the resources in the reference server
1. option to create narrative using jinja2 templates (that is a topic for another notebook)

### import all the modules you need

In [163]:
from fhir_model_generator.model import fhirdate, fhirreference, group, patient, organization, bundle, identifier, humanname, contactpoint, address
from json import dumps
from requests import get, post
from datetime import datetime, date, timedelta
from IPython.display import display as Display, HTML, Markdown
import FHIR_templates as f
from utils import bundle_me, to_json

### Initialize the Group Resource

In [164]:
now = datetime.utcnow().strftime('%Y%m%d%H%M%S%f')
my_g = group.Group(
        dict(
            id = f'group-{pl_use_case}-{now}',
            type = 'person',
            actual = True,
            name = pl_use_case,
            active = True,
            #quantity = df.shape[0],          
        )
) # initialize group instance

print(dumps(my_g.as_json(), indent =4))

{
    "resourceType": "Group",
    "id": "group-active-today-20200505060432348158",
    "active": true,
    "type": "person",
    "actual": true,
    "name": "active-today"
}


### add an identifier and the managingEntity element - i.e. the Organization that created the group.

In [165]:
my_id = identifier.Identifier( # add complex identifier element
    dict(
        value = my_g.id,
        system = "http://example.org",
        ))
my_g.identifier = [my_id]

my_me = fhirreference.FHIRReference(
    dict(
        reference = "Organization/example-organization-2",
        display = "US Core Example Organization 2",
    )
)
my_g.managingEntity = my_me

print(dumps(my_g.as_json(), indent =4))

{
    "resourceType": "Group",
    "id": "group-active-today-20200505060432348158",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "group-active-today-20200505060432348158"
        }
    ],
    "active": true,
    "type": "person",
    "actual": true,
    "name": "active-today",
    "managingEntity": {
        "reference": "Organization/example-organization-2",
        "display": "US Core Example Organization 2"
    }
}


### Add Group Characteristics

- define the parameters to filter by  here based on the use case
- for All patients - no filters
- create a function and run through the parameters to build the element

In [166]:
preface = dict(
        practitioner = ("Doctor ","Practitioner"),
        service = ('Service: ',"HealthcareService"),
        location = ('Location: ',"Location"),
        )

def get_timeframe(timeframe):
    if timeframe == "today":
        start = str(date.today())
        end = str(date.today() + timedelta(days=1))
    elif timeframe == "tomorrow":
        start = str(date.today() + timedelta(days=1))
        end = str(date.today() + timedelta(days=2))
    else:
        start=df.admit_date.min()[:-9]
        end=df.admit_date.max()[:-9]       
    return((start,end))


def get_gc_code(parameter):
    cc = dict(
            text=parameters_map[parameter][2],
            coding=[dict(
                    system = parameters_map[parameter][1],
                    code = parameters_map[parameter][0],
                    display = parameters_map[parameter][2]
                    )
                ]
            )
    return(cc)


def get_characteristic(parameter, v_type, value, start, end):
    print(v_type, value)
    period = dict(
                start = start,
                end = end,
            )
    my_gc = group.GroupCharacteristic(dict
                (
                exclude=False,
                code =  get_gc_code(parameter), #{"text":parameter},
                valueBoolean = True, #v_type default is boolean
                period = period,
                )
            )
    if v_type in ["reference"]:
        my_gc.valueBoolean= None
        my_gc.valueReference = fhirreference.FHIRReference(
                reference = f"{parameters_map[parameter][0]}/{value}",
                 display = f"{parameters_map[parameter][3]} {value.capitalize()}",
            )                                                    
    return(my_gc)
 
                                      
my_g.characteristic=[]
start,end = get_timeframe(timeframe)
                                      
for parameter in parameters: #loop through parameter variables get list of values
    print(parameter, timeframe)
    if parameter in ['service','location','practitioner']:
        v_type = 'reference'
        value = getattr(df, parameters_map[parameter][0]).unique()[0] # all unique parameter values just get first one for now               
    else: # parameter in ['all_patients','discharged','active']:
        v_type = 'boolean'                                   
        value = True
    my_g.characteristic.append(get_characteristic(
        parameter=parameter, v_type = v_type, value = value, start=start, end=end
            ))
print(dumps(my_g.as_json(), indent =4))

active today
boolean True
{
    "resourceType": "Group",
    "id": "group-active-today-20200505060432348158",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "group-active-today-20200505060432348158"
        }
    ],
    "active": true,
    "type": "person",
    "actual": true,
    "name": "active-today",
    "managingEntity": {
        "reference": "Organization/example-organization-2",
        "display": "US Core Example Organization 2"
    },
    "characteristic": [
        {
            "code": {
                "coding": [
                    {
                        "system": "http://loinc.org",
                        "code": "LA16666-2",
                        "display": "Active"
                    }
                ],
                "text": "Active"
            },
            "valueBoolean": true,
            "exclude": false,
            "period": {
                "start": "2020-05-04",
                "end": "2020-05-05"
  

### Finally Add the Group Member from Excel

- for now use the business identifier instead of the FHIR Reference.
- get members based on characteristics defined above

In [167]:
def get_valueref(i):
    valueref =  i.valueReference.reference
    return(valueref.split('/')[1])


def get_filter(i):
    # filter by timeframe
    print(i.period.start.as_json(), i.period.end.as_json(), type(i.period.end.as_json()))
    print(df.admit_date)
    tf_filter = (df.admit_date.astype('str') >= i.period.start.as_json()) & (df.admit_date.astype('str') <= 
                                            f'{i.period.end.as_json()} 00:00:00') # temp fix to align types for comparison
    print(f'tf_filter =\n{tf_filter}')
    char_code = i.code.coding[0].code
    print(parameter)
    if char_code == "LA16666-2":
        p_filter = (df.discharge_date >= i.period.end.as_json())
    if char_code == "LA14102-0":
        p_filter = (df.discharge_date <= i.period.end.as_json())
    if char_code in ['Practitioner','HealthCareService','Location']:
        print(get_valueref(i))
        p_filter = (df[char_code] == get_valueref(i))
    #print( f'p_filter =\n{p_filter}')
    #print(f'p_filter + tf_filter=\n{p_filter & tf_filter}')
    try:
        return(p_filter & tf_filter)
    except UnboundLocalError:
        return(tf_filter)

my_filter = True
for i in my_g.characteristic:
    my_filter = my_filter & get_filter(i)
    #print("my_filter:\n", my_filter)
df[my_filter]

2020-05-04 2020-05-05 <class 'str'>
0    2020-05-03 00:00:00
1    2020-05-03 00:00:00
2    2020-05-03 00:00:00
3    2020-05-04 00:00:00
4    2020-05-04 00:00:00
5    2020-05-04 00:00:00
6    2020-05-04 00:00:00
7    2020-05-05 00:00:00
8    2020-05-05 00:00:00
9    2020-05-05 00:00:00
Name: admit_date, dtype: object
tf_filter =
0    False
1    False
2    False
3     True
4     True
5     True
6     True
7     True
8     True
9     True
Name: admit_date, dtype: bool
active


,row,my_id,f_name,l_name,gender,b_date,is_deceased,is_active,phone,address1,city,state,zip,Practitioner,Location,HealthCareService,admit_date,discharge_date
3,4,456,Daniel,Crimson,M,1996-05-23 00:00:00,False,True,5555555555,100 Yahome St,Napa,CA,94558,jones,St Helena,proctology,2020-05-04 00:00:00,2020-05-05 00:00:00
4,5,567,Elizabeth,Green,F,1941-10-21 00:00:00,False,True,5555555555,100 Trancas Blvd,Napa,CA,94559,smith,Queen,urology,2020-05-04 00:00:00,2020-05-05 00:00:00
5,6,689,Frederick,Gray,M,1964-06-26 00:00:00,False,True,5555555555,100 A St,Napa,CA,94558,jones,St Helena,dermatology,2020-05-04 00:00:00,2020-05-05 00:00:00
6,7,890,Greg,Orange,M,1982-05-01 00:00:00,False,True,5555555555,100 Elm St,Napa,CA,94558,smith,Queen,proctology,2020-05-04 00:00:00,2020-05-06 00:00:00
7,8,901,Harriet,Silver,F,2002-02-02 00:00:00,False,True,5555555555,100 E St,Napa,CA,94558,jones,St Helena,urology,2020-05-05 00:00:00,2020-05-07 00:00:00
8,9,902,Imelda,Gold,F,1959-03-19 00:00:00,False,True,5555555555,100 Imola St,Napa,CA,94558,smith,Queen,dermatology,2020-05-05 00:00:00,2020-05-06 00:00:00
9,10,903,Jack,White,M,1988-08-08 00:00:00,False,True,5555555555,200 Trancas Blvd,Napa,CA,94558,jones,St Helena,proctology,2020-05-05 00:00:00,2020-05-06 00:00:00


In [172]:
my_g.member =[]
for i in df[my_filter].itertuples(index=True):    
    my_member = group.GroupMember(
        dict(
            inactive= False,
            entity = dict(
                identifier = dict(
                        value = i.my_id,
                        system = "http://example.org",
                    ),
                display = f'Name = {i.f_name} {i.l_name} (MRN = {i.my_id})',
            ),
        )
    )
    
    my_g.member.append(my_member)

print(to_json(my_g))

{
  "resourceType": "Group",
  "id": "group-active-today-20200505060432348158",
  "identifier": [
    {
      "system": "http://example.org",
      "value": "group-active-today-20200505060432348158"
    }
  ],
  "active": true,
  "type": "person",
  "actual": true,
  "name": "active-today",
  "managingEntity": {
    "reference": "Organization/example-organization-2",
    "display": "US Core Example Organization 2"
  },
  "characteristic": [
    {
      "code": {
        "coding": [
          {
            "system": "http://loinc.org",
            "code": "LA16666-2",
            "display": "Active"
          }
        ],
        "text": "Active"
      },
      "valueBoolean": true,
      "exclude": false,
      "period": {
        "start": "2020-05-04",
        "end": "2020-05-05"
      }
    }
  ],
  "member": [
    {
      "entity": {
        "identifier": {
          "system": "http://example.org",
          "value": "456"
        },
        "display": "Name = Daniel Crimson (MRN = 

   
### ======= Validate  =======
    

In [173]:
ref_server ='http://hapi.fhir.org/baseR4'
#ref_server = 'http://test.fhir.org/r4'
#ref_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
#ref_server = 'https://vonk.fire.ly/R4'
headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
params = dict(
      )
print('validating ...')
r = post(f'{ref_server}/{my_g.resource_type}/$validate',
         params = params, headers = headers, data = to_json(my_g))

try:
    display(HTML(
        '<h1>Validation output</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<h4>Response</h4><br />'
        f'{r.json()["text"]["div"]}'
        ))
    
except KeyError:
    display(HTML(
        '<h1>Validation output</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<h4>Response</h4><br />'
        f'<pre>{dumps(r.json(),indent=4)}</pre>'
        ))  

validating ...


WARNING,"[Group, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
INFORMATION,"[Group.characteristic[0].code, Line 21, Col 20]","Binding for path Group.characteristic[0].code has no source, so can't be checked"
ERROR,"[Group.characteristic[0].code.coding[0], Line 22, Col 12]","Unknown code {http://loinc.org}LA16666-2 for ""http://loinc.org#LA16666-2"""


### ======= Bundle into Transaction/Save/Load to ref FHIR Server ===========

- Create Transaction bundle using PUT to preserve ids
- Use static example organization from US Core

  

In [174]:
print("......Bundle into transaction ......")

org = organization.Organization(f.example_org)
my_b = bundle_me(ref_server, org)
print(to_json(my_b))
my_b = bundle_me(ref_server,my_g,my_b)
print("......Bundle into transaction add my_g ......")
print(to_json(my_b))

......Bundle into transaction ......
{
  "resourceType": "Bundle",
  "id": "transaction-bundle-20200505061301883989",
  "type": "transaction",
  "timestamp": "2020-05-04T23:13:01.884119",
  "entry": [
    {
      "fullUrl": "http://hapi.fhir.org/baseR4/example-organization-2",
      "resource": {
        "resourceType": "Organization",
        "id": "example-organization-2",
        "meta": {
          "profile": [
            "http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization"
          ]
        },
        "identifier": [
          {
            "system": "http://hl7.org.fhir/sid/us-npi",
            "value": "1407071236"
          },
          {
            "system": "http://example.org/fhir/sid/us-tin",
            "value": "121111111"
          }
        ],
        "active": true,
        "type": [
          {
            "coding": [
              {
                "system": "http://terminology.hl7.org/CodeSystem/organization-type",
                "code": "prov

### ======= POST Bundle to ref FHIR Server ===========

In [175]:
print("......Posting to ref server ......")   
r = post(f'{ref_server}',
          params = params, headers = headers, data = to_json(my_b))
try:
    display(HTML(
        '<h1>Post Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))
except KeyError:
    display(HTML(
        '<h1>Post Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        #f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))   

......Posting to ref server ......


###  Todo:

- save to raw resources to file/ save ids as a dictionary.
- Add Narratives
- Fetch ids for patients to replace the identifiers
- create GET transactions as markdown and text

### Fetch Patient List based on Parameters

- docuement the transaction in markdwon

In [177]:
def pretty_print_request(req):
    """
    At this point it is completely built and ready
    to be fired; it is "prepared".

    However pay attention at the formatting used in 
    this function because it is programmed to be pretty 
    printed and may differ from the actual request.
    """
    req_headers = ('{}\n{}\r\n{}\r\n\r\n{}'.format(
        '-----------START-----------',
        req.method + ' ' + req.url,
        '\r\n'.join('{}: {}'.format(k, v) for k, v in req.headers.items()),
        req.body if req.body else '',
    ))
    return(req_headers)

params = {
    "managing-entity": "example-organization-2",
    #"characteristic-value": "LA18187-7&true", #all
    "characteristic-value": "LA16666-2&true", #active
    #"characteristic-referencevalue": "Practitioner&Practitioner/smith",
    #"characteristic-value": "service&HealthcareService/proctology",
    "timeframe": "2020-05-04", # custom date SP for period
    
        }
print("......Fetching Patient List based on params......")    
r = get(f'{ref_server}/{my_g.resource_type}',
      params = params, headers = headers)

display(HTML(
        '<h1>Request</h1>'
        '<h3>Request Headers: </h3>'
        '<pre>'
        f'{pretty_print_request(r.request)}'
        '</pre>'
        '===============================================<br /><br /><br />'
        )
       )

try:
    display(HTML(
        '<h1>Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))
except KeyError:
    display(HTML(
        '<h1>Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        #f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))
r.json()

......Fetching Patient List based on params......


ERROR,[],"Unknown search parameter ""timeframe"". Value search parameters for this search are: [_id, _language, actual, characteristic, characteristic-value, code, exclude, identifier, managing-entity, member, type, value]"


{'resourceType': 'OperationOutcome',
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">ERROR</td><td>[]</td><td><pre>Unknown search parameter &quot;timeframe&quot;. Value search parameters for this search are: [_id, _language, actual, characteristic, characteristic-value, code, exclude, identifier, managing-entity, member, type, value]</pre></td>\n\t\t\t\t\t\n\t\t\t\t\n\t\t\t</tr>\n\t\t</table>\n\t</div>'},
 'issue': [{'severity': 'error',
   'code': 'processing',
   'diagnostics': 'Unknown search parameter "timeframe". Value search parameters for this search are: [_id, _language, actual, characteristic, characteristic-value, code, exclude, identifier, managing-entity, member, type, value]'}]}